## Felipe Ribeiro




# Bibliotecas

In [19]:
# Instalação das Bibliotecas

!pip install transformers accelerate bitsandbytes PyPDF2 --quiet
!pip install langchain sentence-transformers faiss-cpu --quiet

In [21]:
# Importações e Carregamento do Modelo
import torch
import json
import PyPDF2
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configuração para carregar o modelo com quantização de 4 bits
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Nome do modelo no Hugging Face
model_name = "HuggingFaceH4/zephyr-7b-beta"

# Carrega o tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Carrega o modelo com a configuração de quantização
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto", # Mapeia o modelo automaticamente para a GPU
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import re

def limpar_texto(texto):
    """Remove quebras de linha excessivas e outros artefatos."""
    # Substitui múltiplos espaços/quebras de linha por um único espaço
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

def criar_indice_pesquisavel(texto_pdf):
    """
    Divide o texto do PDF em pedaços, cria embeddings e retorna um índice FAISS.
    """
    # 1. Dividir o texto em pedaços menores (chunks)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # Tamanho de cada pedaço em caracteres
        chunk_overlap=100, # Sobreposição entre pedaços para não perder contexto
        length_function=len
    )
    chunks = text_splitter.split_text(texto_pdf)

    # 2. Criar Embeddings (transformar texto em vetores)
    # Usaremos um modelo mais leve, otimizado para português, para esta tarefa.
    model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
    embeddings = HuggingFaceEmbeddings(model_name=model_name)

    # 3. Criar o índice FAISS a partir dos chunks e embeddings
    # FAISS é uma biblioteca super eficiente para busca de similaridade.
    vector_store = FAISS.from_texts(chunks, embedding=embeddings)
    return vector_store

def buscar_contexto_relevante(indice, reivindicacao, k=3):
    docs_relevantes = indice.similarity_search(reivindicacao, k=k)
    # Concatena o conteúdo dos documentos encontrados
    contexto = "\n---\n".join([doc.page_content for doc in docs_relevantes])
    return contexto

In [23]:
#Prompt + trata o json
def analisar_reivindicacao_com_contexto(reivindicacao, contexto):
    """
    Usa o LLM para analisar UMA ÚNICA reivindicação com base em um contexto específico.
    """
    # Prompt aprimorado com a técnica "few-shot", dando um exemplo claro do que esperamos.
    prompt = f"""<|system|>
Você é um especialista em análise jurídica. Sua tarefa é avaliar se a 'REIVINDICAÇÃO' é suportada pelo 'CONTEXTO' fornecido.
Responda APENAS com um único objeto JSON válido. Não adicione nenhuma explicação ou texto antes ou depois do objeto JSON.

Exemplo de resposta esperada:
{{
  "label": "Incorreta",
  "evidence": "A justificativa para a incorreção, baseada estritamente no contexto."
}}
</s>
<|user|>
'CONTEXTO':
---
{contexto}
---

'REIVINDICAÇÃO':
"{reivindicacao}"

Gere o objeto JSON para a reivindicação acima, baseando-se estritamente no contexto fornecido.</s>
<|assistant|>
"""
    # Prepara a entrada para o modelo
    inputs = tokenizer(prompt, return_tensors="pt", padding=False, truncation=False).to("cuda")

    # Gera a resposta do modelo
    outputs = model.generate(
        **inputs,
        max_new_tokens=300, # Aumentado um pouco para garantir que a 'evidence' caiba
        temperature=0.0,    # Temperatura 0.0 para respostas mais diretas e menos criativas
        do_sample=False,    # Desativa a amostragem para respostas mais determinísticas
        pad_token_id=tokenizer.eos_token_id
    )

    # Decodifica a resposta completa
    resposta_completa = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Lógica de parsing inteligente para extrair o JSON
    try:
        # Pega apenas o que vem depois da tag do assistente
        resposta_assistente = resposta_completa.split("<|assistant|>")[1].strip()

        # Procura pelo primeiro '{' e o último '}' para extrair o bloco JSON
        match = re.search(r'\{.*\}', resposta_assistente, re.DOTALL)
        if match:
            json_str = match.group(0)
            resultado_json = json.loads(json_str)
            return resultado_json
        else:
            # Se não encontrar um JSON, registra o erro
            print(f"ERRO: Bloco JSON não encontrado na saída do modelo para a reivindicação: '{reivindicacao}'")
            print(f"Saída do modelo: {resposta_assistente}")
            return {"label": "Erro", "evidence": "Bloco JSON não encontrado na resposta do modelo."}

    except (json.JSONDecodeError, IndexError) as e:
        print(f"ERRO ao decodificar JSON para a reivindicação: '{reivindicacao}'")
        print(f"Saída do modelo: {resposta_completa}")
        return {"label": "Erro", "evidence": f"Falha ao processar a resposta do modelo: {e}"}

In [24]:
#ler o arquivos
def ler_pdf(caminho_arquivo):
    """Lê o texto de um arquivo PDF."""
    texto = ""
    with open(caminho_arquivo, 'rb') as f:
        leitor = PyPDF2.PdfReader(f)
        for pagina in leitor.pages:
            texto_pagina = pagina.extract_text()
            if texto_pagina:
                texto += texto_pagina
    return texto

def ler_txt(caminho_arquivo):
    """Lê o texto de um arquivo TXT."""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return f.read()


# --- Definição dos arquivos ---
arquivos_para_analisar = [
    {
        "doc_path": "Acórdão 733 de 2025 Plenário.pdf",
        "resumo_path": "Acórdão 733-2025 resumos.txt",
        "doc_name": "Acórdão 733 de 2025 Plenário",
        "summary_id": 1
    },
    {
        "doc_path": "Acórdão 764 de 2025 Plenário.pdf",
        "resumo_path": "Acórdão 764-2025 resumos.txt",
        "doc_name": "Acórdão 764 de 2025 Plenário",
        "summary_id": 2
    }
]

analise_final = []

print("Iniciando a análise dos documentos com a nova estratégia...")

# realizar a análise
for item in arquivos_para_analisar:
    print(f"\n--- Processando: {item['doc_name']} ---")
    try:
        # 1. Ler e limpar os textos
        texto_acordao_bruto = ler_pdf(item['doc_path'])
        texto_acordao = limpar_texto(texto_acordao_bruto)
        texto_resumo = ler_txt(item['resumo_path'])

        # 2. Criar o índice pesquisável para o acórdão
        print("Criando índice de busca para o documento...")
        indice_acordao = criar_indice_pesquisavel(texto_acordao)
        print("Índice criado com sucesso.")

        # 3. Dividir o resumo em reivindicações, ignorando títulos/linhas curtas
        reivindicacoes = [
            r.strip() for r in texto_resumo.split('\n')
            if r.strip() and len(r.strip().split()) > 3 # <-- FILTRO MELHORADO AQUI
        ]
        print(f"Encontradas {len(reivindicacoes)} reivindicações válidas no resumo.")

        # 4. Analisar cada reivindicação individualmente
        for i, claim_text in enumerate(reivindicacoes):
            print(f"Analisando reivindicação {i+1}/{len(reivindicacoes)}: '{claim_text[:50]}...'")

            # 4.1. Buscar contexto relevante no índice
            contexto = buscar_contexto_relevante(indice_acordao, claim_text)

            # 4.2. Chamar o LLM com a reivindicação e o contexto focado
            resultado_analise = analisar_reivindicacao_com_contexto(claim_text, contexto)

            # 4.3. Montar o objeto JSON final para esta reivindicação
            if resultado_analise:
                resultado_final_claim = {
                    "doc_name": item['doc_name'],
                    "claim_text": claim_text,
                    "label": resultado_analise.get('label', 'Erro'),
                    "evidence": resultado_analise.get('evidence', ''),
                    "summary_id": item['summary_id'],
                    "claim_id": i
                }
                analise_final.append(resultado_final_claim)

        print(f"Análise de '{item['doc_name']}' concluída.")

    except FileNotFoundError as e:
        print(f"ERRO: Arquivo não encontrado - {e}. Verifique se fez o upload do arquivo para o Colab.")
    except Exception as e:
        print(f"ERRO inesperado ao processar {item['doc_name']}: {e}")
        import traceback
        traceback.print_exc()


# Salva o resultado final em um único arquivo JSON
if analise_final:
    caminho_saida_json = "analise_reivindicacoes.json"
    with open(caminho_saida_json, 'w', encoding='utf-8') as f:
        json.dump(analise_final, f, ensure_ascii=False, indent=4)

    print(f"\n--- ANÁLISE COMPLETA! ---")
    print(f"Resultado salvo em: {caminho_saida_json}")

    # Imprime o JSON final formatado
    print("\nConteúdo do JSON gerado:")
    print(json.dumps(analise_final, indent=4, ensure_ascii=False))

Iniciando a análise dos documentos com a nova estratégia...

--- Processando: Acórdão 733 de 2025 Plenário ---
Criando índice de busca para o documento...


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Índice criado com sucesso.
Encontradas 3 reivindicações válidas no resumo.
Analisando reivindicação 1/3: 'O processo TC 004.980/2017-4 foi iniciado por inic...'


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Analisando reivindicação 2/3: 'A representação TC 004.980/2017-4, apresentada pel...'


...

Índice criado com sucesso.
Encontradas 3 reivindicações válidas no resumo.
Analisando reivindicação 1/3: 'O processo TC 024.887/2024-2 foi instaurado pelo M...'


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Analisando reivindicação 2/3: 'O processo TC 024.887/2024-2 refere-se a questiona...'


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Analisando reivindicação 3/3: 'No Acórdão 764/2025, o Tribunal de Contas da União...'
Análise de 'Acórdão 764 de 2025 Plenário' concluída.

--- ANÁLISE COMPLETA! ---
Resultado salvo em: analise_reivindicacoes.json

Conteúdo do JSON gerado:
[
    {
        "doc_name": "Acórdão 733 de 2025 Plenário",
        "claim_text": "O processo TC 004.980/2017-4 foi iniciado por iniciativa do próprio BNDES, visando a regularização de sua política salarial diante das novas diretrizes do Ministério da Fazenda. A principal conclusão do TCU foi o reconhecimento de que o BNDES é uma estatal dependente da União, o que implica a imediata aplicação do teto remuneratório constitucional a todos os seus empregados, inclusive nas subsidiárias. O acórdão também determinou que o banco restituísse os valores recebidos a título de PLR nos últimos cinco anos por violar os limites legais de remuneração.",
        "label": "Correta",
        "evidence": "O processo TC 004.980/2017-4 confirmou que o BNDES é uma estata